In [9]:
# !pip install -qU langchain-cli
# !pip install -qU langsmith
# !pip install -qU python-dotenv
# 使用语言模型
# !pip install -qU langchain-openai

In [10]:
import os

from dotenv import load_dotenv

load_dotenv()

# 读取LangSmith Key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "lang-env"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
GPT_GOD_API_KEY = os.getenv("GPT_GOD_API_KEY")


In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="glm-4-flash",
                   base_url="https://api.gptgod.online/v1/",
                   api_key=GPT_GOD_API_KEY
                   )

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]
model.invoke(messages)

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cc65f24d-8fe6-47b4-a637-215391f276fb-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

## 使用简单解析器
如果只是需要简单的AIMessage我们用上述的方式就可以结束了，但是我们往往需要更加具体的操作。接着往下走。

In [13]:
# 引入一个简单的结果解析器
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

使用它的一种方式是单独使用它。例如，我们可以保存语言模型调用的结果，然后将其传递给解析器。

In [14]:
# result = model.invoke(messages)
# parser.invoke(result)

但是这个做法不是很常用，常用的操作我们接着往下看：

In [15]:
chain = model | parser
chain.invoke(messages)

'ciao!'

如果我们现在看一下 LangSmith，我们会发现这个链条有两个步骤：首先调用语言模型，然后将其结果传递给输出解析器。我们可以在 LangSmith 跟踪 中看到这一点。

## Prompt Template

In [16]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [17]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [18]:
chain = prompt_template | model | parser

In [19]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'

## 构建服务
启动服务，将chain暴露为RESTful API

In [ ]:
# !pip install -qU "langserve[all]"

### 创建一个Serve.py 服务脚本